In [0]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
LIST_OF_LABELS = 'daisy,dandelion,roses,sunflowers,tulips'.split(',')
HEIGHT = 299
WIDTH = 299
NUM_CHANNELS = 3
NCLASSES = 5

In [0]:
def linear_model(img, mode, hparams):
  X = tf.reshape(img, [-1, HEIGHT*WIDTH*NUM_CHANNELS]) #Flatten
  ylogits = tf.layers.dense(X,NCLASSES, activation=None)
  return ylogits, NCLASSES

def dnn_model(img, mode, hparams):
  X = tf.reshape(img, [-1, HEIGHT*WIDTH*NUM_CHANNELS]) #Flatten
  h1 = tf.layers.dense(X, 300, activation='tf.nn.relu')
  h2 = tf.layers.dense(h1, 100, activation='tf.nn.relu')
  h3 = tf.layers.dense(h2, 30, activation='tf.nn.relu')
  ylogits = tf.layers.dense(h3, NCLASSES, activation=None)
  
  return ylogits, NCLASSES

def dnn_dropout_model(img, mode, hparams):
  dprob = hparams.get('dprob', 0.1)
  
  X = tf.reshape(img, [-1, HEIGHT*WIDTH*NUM_CHANNELS]) #Flatten
  h1 = tf.layers.dense(X, 300, activation='tf.nn.relu')
  h2 = tf.layers.dense(h1, 100, activation='tf.nn.relu')
  h3 = tf.layers.dense(h2, 30, activation='tf.nn.relu')
  h3d = tf.layers.dropout(h3, rate=dprob, training=(
                              mode ==  tf.estimators.ModeKeys.TRAIN)) #only dropout when training
  ylogits = tf.layers.dense(h3d, NCLASSES, activation=None)
  return ylogits, NCLASSES

def cnn_model(img, mode, hparams):
  ksize1 = hparams.get('ksize', 5)
  ksize2 = hparams.get('ksize2', 5)
  nfil1 =  hparams.get('nfil1', 10)
  nfil2 =  hparams.get('nfil2', 20)
  dprob =  hparams.get('dprob', 0.25)
  
  c1 = tf.layers.conv2d(img, filters=nfil1,
                        kernel_size=ksize1,
                        strides=1,
                        padding='same,
                        activation=tf.nn.relu)
  p1 = tf.layers.max_pooling2d(c1, pool_size=2, strides=2)

  c2 = tf.layers.conv2d(img, filters=nfil2,
                        kernel_size=ksize2,
                        strides=1,
                        padding='same,
                        activation=tf.nn.relu)
  p1 = tf.layers.max_pooling2d(c2, pool_size=2, strides=2)

  outlen = p2.shape[1]*p2.shape[2]*p2.shape[3]
  p2flat = tf.reshape(p2, [-1, outlen]) #Flatten
  
  #apply batch normalization
  if hparams['batch_norm']:
    h3 = tf.layers.dense(p2flat, 300, activation=None)
    h3 = tf.layers.batch_normalization(
          h3, training=(mode == tf.estimator.ModeKeys.TRAIN)) #Only batch norm in training
    h3 = tf.layers.relu(h3)
  else:
    h3 = tf.layers.dense(p2flat, 300, activation=tf.nn.relu)
    
  #apply dropout
  h3d = tf.layers.dropout(h3, rate=dropout, training = (mode == tf.estimator.ModeKeys.TRAIN))
  ylogits = tf.layers.dense(p2flat, 300, activation=tf.nn.relu)
  #apply batch normalization
  if hparams['batch_norm']:
    ylogits = tf.layers.batch_normalization(
        ylogits, training = (mode = tf.estimator.ModeKeys.TRAIN))
  return ylogits, NCLASSES